In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet


path='D://4th Semester//Data Mining2//DMC//Clustering//'
df_cluster = pd.read_csv(path+'clusters_v3.csv',usecols=['pid','size','Cluster'])
df_train = pd.read_csv(path+'train_pivot_table.csv',usecols=[*range(0,94)])

'''
#remove "." in the cluster table
pid_n=[]
for i in range (0, len(df_cluster)):
    s=df_cluster['pid'].iloc[i]
    s=str(s).split('.')[0]
    s=int(s)
    pid_n.append(s)

df_cluster.insert(1,column='pid_n',value=pid_n)
df_cluster.drop(['pid'], axis=1,inplace=True)
df_cluster.rename(columns={'pid_n':'pid'}, inplace=True)
df_cluster.head()

df_cluster.to_csv(path+'clusters_v1.csv',index=False)
'''
def outlier_label(df_cluster):
    for i in range(0,len(df_cluster)):
        if df_cluster['Cluster'].iloc[i]==-1:
            a=1
            df_cluster.set_value(i, 'Cluster', -1*(i+1), takeable=False)
            #df_cluster['Cluster'].iloc[i]=-(i+1)
    return df_cluster

#give different labels to outliers, which used to be -1
#df_cluster=outlier_label(df_cluster)


In [2]:

#df_train.join(df_cluster.set_index(['key','size']), on=['pid','size'])
df=pd.merge(df_train, df_cluster, on=['pid', 'size'])
df.drop(['pid','size'], axis=1, inplace=True)

date = pd.read_csv(path+'daily_avg.csv',usecols=['Date'])
i=31
while i>0:
    date.drop(date.index[[91+i]],inplace=True) #delete January
    i-=1
date=np.array(date['Date']) #list of dates Oct-Dec
print(len(date))

path_test = 'D://4th Semester//Data Mining2//DMC//Test//'
test = pd.read_csv(path_test+'test_0.csv')
test = pd.merge(test, df_cluster, on=['pid', 'size'])

print(len(set(test['Cluster'])))
print(set(test['Cluster']))

92
52
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, -1}


In [3]:
#function to calculate sum(d2-d1) for each cluster
def calculate_performance(pred, cluster):
    """
    Calculates first day that stock hits 0 in a certain month for an item
    :param pred: Array of predicted sales units for an item
    :param cluster: cluster of the item
    :return: sum(d2-d1)
    """
    print('Calculating...')
    path='D://4th Semester//Data Mining2//DMC//Clustering//'
    df_cluster = pd.read_csv(path+'clusters_v3.csv',usecols=['pid','size','Cluster'])
    #df_cluster=outlier_label(df_cluster)
    path_test = 'D://4th Semester//Data Mining2//DMC//Test//'
    test = pd.read_csv(path_test+'test_0.csv')
    test = pd.merge(test, df_cluster, on=['pid', 'size'])
    df=test[test['Cluster'] == cluster]
    #sold_out_date=test['sold_out_date'][test['Cluster'] == cluster]
    sum_gap=0
    for i in range(len(df)):
        this_stock=df['stock'].iloc[i]
        d_real=df['sold_out_date'].iloc[i].split('-')[2]
        d_real=int(d_real)
        for day in range(len(pred)):
            this_stock -= pred[day]
            if this_stock <= 0:
                soldout_day = day+1
                break
        if this_stock > 0:
            soldout_day = 31
        sum_gap+=abs(soldout_day-d_real)                
    return sum_gap

'''
#try function
df_this_cluster=df[df.Cluster == 12]
daily_avg=np.array(df_this_cluster.mean())
#remove the last column as cluster label
daily_avg=daily_avg[0:92]
fb_this_cluster={'ds': date, 'y': daily_avg}
fb_this_cluster=pd.DataFrame(data=fb_this_cluster)
        
#apply fb prophet
m = Prophet()
m.fit(fb_this_cluster)
future = m.make_future_dataframe(periods=31)
forecast = m.predict(future)
forecast_this_cluster = forecast[['ds', 'yhat']]

f=np.array(forecast_this_cluster['yhat'])
a=calculate_performance(f,12)
a
'''
####################################
#forecasting

# Taken from Basil: datamining2/data/datasets_Basil/create_dataset_v0.3.ipynb
#creating array with dates where 11 Teamsports announces discount on social media
marketingactivities=['2017-10-11','2017-10-16','2017-11-04','2017-11-11','2017-11-23','2017-11-24',
                     '2017-11-25','2017-11-27','2017-12-03','2017-12-27','2017-12-28','2017-12-31',
                    '2018-01-14','2018-01-22','2018-01-23','2018-01-30','2018-02-06','2018-02-07',
                     '2018-02-20','2018-02-22','2018-02-23']
marketingactivities = pd.DataFrame({
    'holiday': 'marketingactivity',
    'ds': pd.to_datetime(marketingactivities),
    })


#set(test['Cluster'])
count=1
total_sum_gap=0
for i in set(test['Cluster']):
        df=df.fillna(0)
        df_this_cluster=df[df.Cluster == i]
        daily_avg=np.array(df_this_cluster.mean())
        #remove the last column as cluster label
        daily_avg=daily_avg[0:92]
        fb_this_cluster={'ds': date, 'y': daily_avg}
        fb_this_cluster=pd.DataFrame(data=fb_this_cluster)
        fb_this_cluster=fb_this_cluster.fillna(0)
        
        #apply fb prophet
        ##Prophet(holidays = marketingactivities_prophet).fit(val)
        m = Prophet(changepoint_prior_scale=0.0001,yearly_seasonality=False,holidays=marketingactivities)
        forecast = m.fit(fb_this_cluster)
        future = m.make_future_dataframe(periods=31)
        forecast = m.predict(future)
        
        #edit the forecast to fit the function
        forecast_this_cluster = forecast[['ds', 'yhat']].iloc[92:]
        f=forecast_this_cluster['yhat']
        f=f.fillna(0)
        f=np.array(f)
        for x in range(0,len(f)):
            if f[x] < 0:
                f[x]=0
        sum_gap=calculate_performance(f,i)
        total_sum_gap+=sum_gap
        print('this cluster is')
        print(i)
        print('the error of this cluster is')
        print(sum_gap)
        print('the total error now is:')
        print(total_sum_gap)
        print('count')
        print(count)
        count+=1
print('finish')
total_sum_gap


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\Adele\Anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Calculating...
this cluster is
0
the error of this cluster is
11769
the total error now is:
11769
count
1


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Calculating...
this cluster is
1
the error of this cluster is
1044
the total error now is:
12813
count
2


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Calculating...
this cluster is
49
the error of this cluster is
152
the total error now is:
67241
count
50


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Calculating...
this cluster is
50
the error of this cluster is
127
the total error now is:
67368
count
51
Calculating...
this cluster is
-1
the error of this cluster is
11864
the total error now is:
79232
count
52
finish


79232

In [4]:
import math
math.sqrt(total_sum_gap)

281.4817933721469